In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import lightgbm as lgb
import xgboost as xgb
from sklearn.decomposition import PCA,KernelPCA

In [2]:
train_forecloosure = pd.read_csv("../input/train_foreclosure.csv")
test_foreclosure = pd.read_csv("../input/test_foreclosure.csv")
RF_Final_Data = pd.read_excel("../input/RF_Final_Data.xlsx")
LMS = pd.read_excel("../input/LMS_31JAN2019.xlsx")
customers = pd.read_excel("../input/Customers_31JAN2019.xlsx")

## Lets Find Missing Values

In [3]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [4]:
# Missing values statistics of Customers
missing_values = missing_values_table(train_forecloosure)
missing_values.head(10)

Your selected dataframe has 2 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [5]:
# Missing values statistics of Customers
missing_values = missing_values_table(train_forecloosure)
missing_values.head(10)

Your selected dataframe has 2 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [6]:
# Missing values statistics of Customers
missing_values = missing_values_table(RF_Final_Data)
missing_values.head(10)

Your selected dataframe has 9 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
Preprocessed_Subject,183,2.4
Preprocessed_EmailBody,155,2.1


In [7]:
# Missing values statistics of Customers
missing_values = missing_values_table(LMS)
missing_values.head(10)

Your selected dataframe has 38 columns.
There are 10 columns that have missing values.


,Missing Values,% of Total Values
NPA_IN_LAST_MONTH,610891,97.9
NPA_IN_CURRENT_MONTH,610814,97.8
LAST_RECEIPT_AMOUNT,12045,1.9
LAST_RECEIPT_DATE,3787,0.6
CUSTOMERID,516,0.1
SCHEMEID,516,0.1
DPD,31,0.0
CITY,8,0.0
CURRENT_TENOR,5,0.0
BALANCE_TENURE,5,0.0


In [8]:
# Missing values statistics of Customers
missing_values = missing_values_table(customers)
missing_values.head(10)

Your selected dataframe has 15 columns.
There are 10 columns that have missing values.


,Missing Values,% of Total Values
PROFESSION,10000,100.0
OCCUPATION,10000,100.0
POSITION,9339,93.4
PRE_JOBYEARS,7625,76.2
QUALIFICATION,603,6.0
AGE,396,4.0
MARITAL_STATUS,396,4.0
SEX,394,3.9
NO_OF_DEPENDENT,79,0.8
BRANCH_PINCODE,73,0.7


## Lets adjust the LMS DataFrame

In [9]:
LMS.columns

Index(['AGREEMENTID', 'CUSTOMERID', 'LOAN_AMT', 'NET_DISBURSED_AMT',
       'INTEREST_START_DATE', 'CURRENT_ROI', 'ORIGNAL_ROI', 'CURRENT_TENOR',
       'ORIGNAL_TENOR', 'DUEDAY', 'AUTHORIZATIONDATE', 'CITY',
       'PRE_EMI_DUEAMT', 'PRE_EMI_RECEIVED_AMT', 'PRE_EMI_OS_AMOUNT',
       'EMI_DUEAMT', 'EMI_RECEIVED_AMT', 'EMI_OS_AMOUNT', 'EXCESS_AVAILABLE',
       'EXCESS_ADJUSTED_AMT', 'BALANCE_EXCESS', 'NET_RECEIVABLE',
       'OUTSTANDING_PRINCIPAL', 'PAID_PRINCIPAL', 'PAID_INTEREST',
       'MONTHOPENING', 'LAST_RECEIPT_DATE', 'LAST_RECEIPT_AMOUNT', 'NET_LTV',
       'COMPLETED_TENURE', 'BALANCE_TENURE', 'DPD', 'FOIR', 'PRODUCT',
       'SCHEMEID', 'NPA_IN_LAST_MONTH', 'NPA_IN_CURRENT_MONTH', 'MOB'],
      dtype='object')

In [10]:
LMS=LMS.groupby('AGREEMENTID').agg({      # find the sum of the durations for each group
                                     'CUSTOMERID': 'last',
                                  'LOAN_AMT':'last',
                                  'NET_DISBURSED_AMT':'last',
                                  'INTEREST_START_DATE':'last',
                                  'CURRENT_ROI':'last',  #Could me mean
                                  'ORIGNAL_ROI':'last',
                                  'CURRENT_TENOR':'last',
                                  'ORIGNAL_TENOR':'last',
                                  'DUEDAY':'last',
                                  'AUTHORIZATIONDATE':'last',
                                  'CITY':'last',
                                  'PRE_EMI_DUEAMT':sum,
                                  'PRE_EMI_RECEIVED_AMT':sum,
                                  'PRE_EMI_OS_AMOUNT':sum,
                                  'EMI_DUEAMT':sum,
                                  'EMI_RECEIVED_AMT':sum,
                                  'EMI_OS_AMOUNT':sum,
                                  'EXCESS_AVAILABLE':sum,
                                  'EXCESS_ADJUSTED_AMT':sum,
                                  'BALANCE_EXCESS':sum,
                                  'NET_RECEIVABLE':sum,
                                  'OUTSTANDING_PRINCIPAL':'last',
                                  'PAID_PRINCIPAL':sum,
                                  'PAID_INTEREST':sum,
                                  'MONTHOPENING':'last',
                                  'LAST_RECEIPT_DATE':'last',
                                  'LAST_RECEIPT_AMOUNT':sum,
                                  'NET_LTV':'last',
                                  'COMPLETED_TENURE':'last',
                                  'BALANCE_TENURE':'last',
                                  'DPD':'last',
                                  'FOIR':'last',
                                  'PRODUCT':'last',
                                  'SCHEMEID':'last',
                                  'NPA_IN_LAST_MONTH':'last',
                                  'NPA_IN_CURRENT_MONTH':'last',
                                  'MOB':'last'  
                                })    

In [11]:
missing_values = missing_values_table(LMS)
missing_values.head(10)

Your selected dataframe has 37 columns.
There are 6 columns that have missing values.


,Missing Values,% of Total Values
NPA_IN_LAST_MONTH,20338,61.0
NPA_IN_CURRENT_MONTH,20303,60.9
CUSTOMERID,459,1.4
SCHEMEID,459,1.4
LAST_RECEIPT_DATE,128,0.4
DPD,1,0.0


## Lets Merge the Customer and LMS dataframes

In [12]:
customers.shape

(10000, 15)

In [13]:
LMS=LMS.reset_index()

In [14]:
RF_Final_Data.shape

(7511, 9)

In [15]:
Combine_df = pd.merge(LMS, customers, on='CUSTOMERID',how='outer')

In [16]:
Combine_df.shape

(33354, 52)

In [17]:
Combine_df = pd.merge(Combine_df, train_forecloosure, on='AGREEMENTID',how='outer')

In [18]:
# Missing values statistics of Customers
missing_values = missing_values_table(Combine_df)
missing_values.head(20)

Your selected dataframe has 53 columns.
There are 21 columns that have missing values.


,Missing Values,% of Total Values
PROFESSION,33354,100.0
OCCUPATION,33354,100.0
POSITION,32693,98.0
PRE_JOBYEARS,30979,92.9
QUALIFICATION,23957,71.8
AGE,23750,71.2
MARITAL_STATUS,23750,71.2
SEX,23748,71.2
NO_OF_DEPENDENT,23433,70.3
BRANCH_PINCODE,23427,70.2


## Lets Merge Combined DB with RF Final

In [19]:
RF_Final_Data['Type'].unique()

array(['REQUEST', 'General', 'QUERY', 'Unassigned', 'COMPLAINT',
       'Request', 'ROI', 'Foreclosure'], dtype=object)

In [20]:
RF_Final_Data['SubType'].unique()

array(['PROVISIONAL IT CERTIFICATE', 'SOA', 'Others', 'CHANGEOFROI',
       'TDSREIMBRELATED', 'FORECLOSURE', 'REPAYMENT SCHEDULE',
       'FINAL IT CERTIFICATE', 'LISTOFDOCS', 'Unassigned',
       'PREPAYEMICHNG', 'Final Income Tax Certificate',
       'FINAL IT CERTIFICATE NONHL', 'COPYOFORIGDOCS', 'PREPAYTENORCHNG',
       'CHANGEREPAYMODE', 'CHANGEADDRESS', 'EMIPOSTCLOSURE',
       'UPDATECONTACTDET', 'Statement of Account',
       'PROVISIONAL IT CERTIFICATE NONHL', 'WRONGDEBITEMI',
       'INSURPOLNOTRECD', 'STATUSOFLOANAPP', 'REFOFINSUR', 'NONRECOFINSU',
       'NONRECTLISTDOCS', 'ORIGDOCNOTRECD', 'Repayment Schedule',
       'WRONGCONTACTDET', 'EMINOTBANKED', 'DOCSUBMITFORLOAN', 'COMPLAINT',
       'EMI NOT BANKED', 'NO LIEN LETTER REQUIRED',
       'ORG PROPERTY PAPERS REQUIRED POST LOAN CLOSURE',
       'PAN NO REQUIRED', 'EMI TO BE DEBITED', 'TOP UP LOAN REQUIRED',
       'EXCESS EMI REFUND', 'CHANGE IN THE EMI DUE DATE',
       'NEFT DETAILS REQ', 'ROC CHARGE REMOVAL', 'NOC

In [21]:
RF_Final_Data.shape

(7511, 9)

In [22]:
RF_Final_Data['Masked_CustomerID'].nunique()

2709

In [23]:
Email_Data = RF_Final_Data[['Type','SubType','Masked_CustomerID']]

In [24]:
Email_Data = Email_Data.rename(columns={'Masked_CustomerID': 'CUSTOMERID'})

In [25]:
Email_Data = pd.get_dummies(Email_Data)

In [26]:
Email_Data.columns

Index(['CUSTOMERID', 'Type_COMPLAINT', 'Type_Foreclosure', 'Type_General',
       'Type_QUERY', 'Type_REQUEST', 'Type_ROI', 'Type_Request',
       'Type_Unassigned', 'SubType_CHANGE IN THE EMI DUE DATE',
       'SubType_CHANGEADDRESS', 'SubType_CHANGEOFROI',
       'SubType_CHANGEREPAYMODE', 'SubType_CIBILTOBEUPDATED',
       'SubType_COMPLAINT', 'SubType_COPYOFORIGDOCS',
       'SubType_DISBURSEMENT BREAK UP REQUIRED', 'SubType_DOCSUBMITFORLOAN',
       'SubType_EMI NOT BANKED', 'SubType_EMI TO BE DEBITED',
       'SubType_EMINOTBANKED', 'SubType_EMIPOSTCLOSURE',
       'SubType_EXCESS EMI REFUND', 'SubType_FINAL IT CERTIFICATE',
       'SubType_FINAL IT CERTIFICATE NONHL', 'SubType_FORECLOSURE',
       'SubType_Final Income Tax Certificate', 'SubType_Foreclosure Approval',
       'SubType_INSURPOLNOTRECD', 'SubType_LISTOFDOCS',
       'SubType_NEFT DETAILS REQ', 'SubType_NO LIEN LETTER REQUIRED',
       'SubType_NOC REISSUE', 'SubType_NONRECOFINSU',
       'SubType_NONRECTLISTDOCS',


In [27]:
Email_Data_groupby=Email_Data.groupby('CUSTOMERID').sum()

In [28]:
Email_Data_groupby = Email_Data_groupby.reset_index()

In [29]:
Email_Data_groupby.head()

,CUSTOMERID,Type_COMPLAINT,Type_Foreclosure,Type_General,Type_QUERY,Type_REQUEST,Type_ROI,Type_Request,Type_Unassigned,SubType_CHANGE IN THE EMI DUE DATE,SubType_CHANGEADDRESS,SubType_CHANGEOFROI,SubType_CHANGEREPAYMODE,SubType_CIBILTOBEUPDATED,SubType_COMPLAINT,SubType_COPYOFORIGDOCS,SubType_DISBURSEMENT BREAK UP REQUIRED,SubType_DOCSUBMITFORLOAN,SubType_EMI NOT BANKED,SubType_EMI TO BE DEBITED,SubType_EMINOTBANKED,SubType_EMIPOSTCLOSURE,SubType_EXCESS EMI REFUND,SubType_FINAL IT CERTIFICATE,SubType_FINAL IT CERTIFICATE NONHL,SubType_FORECLOSURE,SubType_Final Income Tax Certificate,SubType_Foreclosure Approval,SubType_INSURPOLNOTRECD,SubType_LISTOFDOCS,SubType_NEFT DETAILS REQ,SubType_NO LIEN LETTER REQUIRED,SubType_NOC REISSUE,SubType_NONRECOFINSU,SubType_NONRECTLISTDOCS,SubType_ORG PROPERTY PAPERS REQUIRED POST LOAN CLOSURE,SubType_ORG SHARE CERTIFICATE REQUIRED FOR TRANSFER,SubType_ORIGDOCNOTRECD,SubType_Others,SubType_PAN NO REQUIRED,SubType_PREPAYEMICHNG,SubType_PREPAYTENORCHNG,SubType_PROVISIONAL IT CERTIFICATE,SubType_PROVISIONAL IT CERTIFICATE NONHL,SubType_REFOFINSUR,SubType_REPAYMENT SCHEDULE,SubType_ROC CHARGE REMOVAL,SubType_ROI Approval,SubType_Repayment Schedule,SubType_SOA,SubType_STATUSOFLOANAPP,SubType_Statement of Account,SubType_TDSREIMBRELATED,SubType_TOP UP LOAN REQUIRED,SubType_UPDATECONTACTDET,SubType_Unassigned,SubType_WELKITNOTRECD,SubType_WRONGCONTACTDET,SubType_WRONGDEBITEMI
0,12001002,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,12001003,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,12001005,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,12001009,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,12001010,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
Email_Data_groupby['CUSTOMERID'].nunique()

2709

In [31]:
Combine_df_with_RF_Data = pd.merge(Combine_df, Email_Data_groupby, on='CUSTOMERID',how='outer')

In [32]:
Combine_df_with_RF_Data.head()

,AGREEMENTID,CUSTOMERID,LOAN_AMT,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,DUEDAY,AUTHORIZATIONDATE,CITY,PRE_EMI_DUEAMT,PRE_EMI_RECEIVED_AMT,PRE_EMI_OS_AMOUNT,EMI_DUEAMT,EMI_RECEIVED_AMT,EMI_OS_AMOUNT,EXCESS_AVAILABLE,EXCESS_ADJUSTED_AMT,BALANCE_EXCESS,NET_RECEIVABLE,OUTSTANDING_PRINCIPAL,PAID_PRINCIPAL,PAID_INTEREST,MONTHOPENING,LAST_RECEIPT_DATE,LAST_RECEIPT_AMOUNT,NET_LTV,COMPLETED_TENURE,BALANCE_TENURE,DPD,FOIR,PRODUCT,SCHEMEID,NPA_IN_LAST_MONTH,NPA_IN_CURRENT_MONTH,MOB,CUST_CONSTTYPE_ID,CUST_CATEGORYID,...,SubType_EMI TO BE DEBITED,SubType_EMINOTBANKED,SubType_EMIPOSTCLOSURE,SubType_EXCESS EMI REFUND,SubType_FINAL IT CERTIFICATE,SubType_FINAL IT CERTIFICATE NONHL,SubType_FORECLOSURE,SubType_Final Income Tax Certificate,SubType_Foreclosure Approval,SubType_INSURPOLNOTRECD,SubType_LISTOFDOCS,SubType_NEFT DETAILS REQ,SubType_NO LIEN LETTER REQUIRED,SubType_NOC REISSUE,SubType_NONRECOFINSU,SubType_NONRECTLISTDOCS,SubType_ORG PROPERTY PAPERS REQUIRED POST LOAN CLOSURE,SubType_ORG SHARE CERTIFICATE REQUIRED FOR TRANSFER,SubType_ORIGDOCNOTRECD,SubType_Others,SubType_PAN NO REQUIRED,SubType_PREPAYEMICHNG,SubType_PREPAYTENORCHNG,SubType_PROVISIONAL IT CERTIFICATE,SubType_PROVISIONAL IT CERTIFICATE NONHL,SubType_REFOFINSUR,SubType_REPAYMENT SCHEDULE,SubType_ROC CHARGE REMOVAL,SubType_ROI Approval,SubType_Repayment Schedule,SubType_SOA,SubType_STATUSOFLOANAPP,SubType_Statement of Account,SubType_TDSREIMBRELATED,SubType_TOP UP LOAN REQUIRED,SubType_UPDATECONTACTDET,SubType_Unassigned,SubType_WELKITNOTRECD,SubType_WRONGCONTACTDET,SubType_WRONGDEBITEMI
0,11220001,12001000.0,1.171011e+07,1.171011e+07,2010-08-30,13.421934,10.612692,45.0,120,1,2010-08-29,MUMBAI,55959.915122,55959.915122,0.0,5.629871e+07,5.520051e+07,1.098200e+06,7.569304e+05,7.569304e+05,0.00000,1.098200e+06,1.164472e+05,6.987372e+07,1.871981e+07,8.693575e+06,2014-05-05,1501354.00,40.06,45,0.0,0.0,0.6000,HL,10901100.0,NaN,NaN,45,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11220002,12001001.0,1.929025e+07,1.929025e+07,2010-09-15,13.734072,10.612692,137.0,180,1,2010-09-15,MUMBAI,100504.575864,100504.575864,0.0,1.061903e+07,1.033958e+07,2.794481e+05,0.000000e+00,0.000000e+00,0.00000,2.794481e+05,1.234760e+07,6.942656e+06,4.945154e+06,1.713861e+07,2013-11-01,126530.00,84.31,38,99.0,0.0,0.6000,HL,10901100.0,NaN,NaN,38,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11220003,12001002.0,4.053499e+06,4.053499e+06,2010-09-06,17.167590,12.485520,360.0,180,5,2010-09-06,MUMBAI,0.000000,0.000000,0.0,2.055559e+08,2.055559e+08,0.000000e+00,3.139265e+06,3.139033e+06,231.89796,-2.318980e+02,3.917012e+06,7.461579e+06,1.322681e+08,3.917012e+06,2015-08-05,1424381.21,79.34,59,301.0,0.0,0.9100,HL,10901101.0,0,0,59,1.0,7.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11220004,12001003.0,1.325703e+07,1.325703e+07,2010-09-29,15.606900,10.924830,198.0,180,5,2010-09-29,MUMBAI,28441.732656,28441.732656,0.0,7.211965e+06,7.211965e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000e+00,1.030278e+07,2.954253e+06,4.219464e+06,1.255472e+07,2013-11-05,88256.00,86.86,37,161.0,0.0,0.9686,HL,10901101.0,NaN,NaN,38,3.0,4.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11220005,12001004.0,1.321140e+07,1.321140e+07,2010-09-06,16.543314,11.861244,142.0,180,5,2010-09-06,MUMBAI,0.000000,0.000000,0.0,7.671444e+07,7.671444e+07,0.000000e+00,1.014741e+06,1.014741e+06,0.00000,0.000000e+00,8.110481e+06,4.422800e+07,3.764543e+07,1.141333e+07,2014-07-05,827028.00,40.93,46,96.0,0.0,

In [33]:
Combine_df_with_RF_Data.shape

(33354, 111)

## We now have our DF , Lets try and delete missing columns and create train and test data

In [34]:
missing_values = missing_values_table(Combine_df_with_RF_Data)
missing_values.head(100)

Your selected dataframe has 111 columns.
There are 79 columns that have missing values.


,Missing Values,% of Total Values
PROFESSION,33354,100.0
OCCUPATION,33354,100.0
POSITION,32693,98.0
PRE_JOBYEARS,30979,92.9
SubType_EMI TO BE DEBITED,30645,91.9
SubType_NONRECOFINSU,30645,91.9
SubType_Others,30645,91.9
SubType_ORIGDOCNOTRECD,30645,91.9
SubType_ORG SHARE CERTIFICATE REQUIRED FOR TRANSFER,30645,91.9
SubType_ORG PROPERTY PAPERS REQUIRED POST LOAN CLOSURE,30645,91.9


In [35]:
Combine_df_with_RF_Data = Combine_df_with_RF_Data.drop(['PROFESSION','OCCUPATION','PRE_JOBYEARS','POSITION'],axis=1)
Combine_df= Combine_df.drop(['PROFESSION','OCCUPATION','PRE_JOBYEARS','POSITION'],axis=1)

In [36]:
train_df = Combine_df[Combine_df['FORECLOSURE'].notnull()]

In [37]:
test_df = Combine_df[Combine_df['FORECLOSURE'].isnull()]

In [38]:
train_df_with_RF = Combine_df_with_RF_Data[Combine_df_with_RF_Data['FORECLOSURE'].notnull()]
test_df_with_RF = Combine_df_with_RF_Data[Combine_df_with_RF_Data['FORECLOSURE'].isnull()]

In [39]:
train_df.shape

(20012, 49)

In [40]:
test_df.shape

(13342, 49)

In [41]:
train_df_with_RF.shape

(20012, 107)

In [42]:
test_df_with_RF.shape

(13342, 107)

In [43]:
train_df.to_csv('train_df.csv',index=False)
test_df.to_csv('test_df.csv',index=False)

In [44]:
train_df_with_RF.to_csv('train_df_with_RF.csv',index=False)
test_df_with_RF.to_csv('test_df_with_RF.csv',index=False)